In [1]:
import json
import nltk 
import string
import copy
import random
from os import listdir
import pickle



import numpy as np

import string
import re

from nltk.stem.porter import *
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier,LinearRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score


from pos_tagger import tag

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# there are four files with dataset, because I collected data in different times
# they are dictionaries, keys are links to news and text of the news is value

text_store=open("article_text_pos_1063_57531copy.json","r")
article_text_dict_positive = json.load(text_store)
text_store.close()
# '3815', "text"

# new negative samples from BBC are :
text_store=open("article_text_neg_bbc_iter1_07272.json","r")
iter1_BBC_text_dict_neg = json.load(text_store)
text_store.close()

text_store=open("iter2_text_neg_bbc_12981.json","r")
iter2_BBC_text_dict_neg = json.load(text_store)
text_store.close()

text_store=open("iter1_article_text_neg_CNN_08109.json","r")
iter1_CNN_neg_text = json.load(text_store)
text_store.close()




In [3]:
toremove=[]
for asd in article_text_dict_positive.items():
    if len(asd[1])<100:
        toremove.append(asd[0])
for keys in toremove:
    del article_text_dict_positive[keys]

In [4]:
print(len(iter1_BBC_text_dict_neg))
print(len(iter2_BBC_text_dict_neg))
print(len(iter1_CNN_neg_text))
print(len(article_text_dict_positive))

1038
1043
948
1037


In [5]:
#those stop words to be removed
my_stopwords = stopwords.words('turkish')

# those are features that are not related to hate-speech so they will be removed
my_stopwords.extend(['facebook', 'telif','hakkı', 'telif hakkı','dr','bbc','bır','dan', 'den',
                    'karsı','twitter',"caption","image",
                    "image caption","tiklayin",'copyright',
                     'yayın','sayfa','no',
                     "tüm","hakları",'saklıdır','copyright'
                    "sayfa","world","world service","yeni akit","gazetesihalkalı",
                  "tüm hakları saklıdır","hakları saklıdır","hakları",'saklıdır',
                    "yayın","sayfa","no",'ve',"cnn","aa","destekyeniakitcomtr","httpyeniakitcomtr"])

# some websites tends to use special characters too much, remove them all
removethose='“’‘”•.,\'\"!;@?())'
removethose+='*'+ '0'+ ':'+ ']'+ '_'+'$'+ '['+'{'+ '}'+ '»'
removethose+='©'+'-'+"1234567890"+"&"+"—"+"/"+"|"+"="+">"+"…"+"%"+"′"

removethose=['“', '’', '‘', '”', '•', '.', ',', "'", '"', '!', 
             ';', '@', '?', '(', ')', ')', '*', '0', ':', ']',
             '_', '$', '[', '{', '}', '»', '©', '-', '1', '2', 
             '3', '4', '5', '6', '7', '8', '9', '0', '&', '—', 
             '/', '|', '=', '>', '…', '%', '′','€', '¥','£','›',
             '¼','<','¨','‏','­','–','#','+']

lowercase=' abcdefghijklmnoprstuvyzğöıüşç'
# Turkish stemmer from https://github.com/otuncelli/turkish-stemmer-python
stemmer = TurkishStemmer()

# this function lowers text, 
# then removes characters such as @ or » because
# those are used in some resources of news more than others and
# also corrects some characters such as 'i̇' which
# does not exists in Turkish but found in the text
# due to encoding or scraping errors
def pre_process_clean_old(text):
    text=text.lower()
    text=text.strip()
    for rmt in removethose:
        text=text.replace(rmt,"")
    text=text.replace('i̇','i').replace('î','i').replace('â','a').replace('á','a')
    text=text.replace('ū','ü').replace('û','u')
    text=text.replace('è','e').replace('é','e').replace('ê','e')
    return text

def pre_process_clean(text):
    text=text.lower()
    text=text.strip()
    text=text.replace('i̇','i').replace('î','i').replace('â','a').replace('á','a')
    text=text.replace('ū','ü').replace('û','u')
    text=text.replace('è','e').replace('é','e').replace('ê','e')
    result=""
    for thechar in text:
        if (thechar in lowercase):
            result+=thechar
    return result

def pre_process_stopwords(text):
    text=nltk.word_tokenize(text.lower().strip())
    text=filter(lambda x: x not in my_stopwords,text )
    text=" ".join(text)
    return text

# There are different tokenization functions

# first one is tokenizes with nltk, lowers text 
# and applies stemming to each word
def tokenize_stem(text):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems text. Returns a list of them"""
    text=nltk.word_tokenize(text.lower().strip())
    tokens = [stemmer.stem(t) for t in text]
    return tokens

# lowers text, uses tagging library's tokenization 
# NO-NEED
# def tokenize(text):
#     text=text.lower().strip()
#     tokens=[]
#     tags=tag(text)
#     for a_tag in tags:
#         if a_tag[0] not in my_stopwords:
#             tokens.append(a_tag[0])
#     return tokens

# lowers text
# generates part of speech tags  
# returns "word+tag" 
# tokenization made with tagging library
def pos_tokenize(text):
    text=text.lower().strip()
    tokens=[]
    tags=tag(text)
    for a_tag in tags:
        tokens.append(a_tag[0]+"+"+a_tag[1])
    return tokens

# lowers text
# splits text to it's characters
def char_tokenize(text):
    text = text.lower().strip()
    tokens = [t for t in text]
    return tokens



In [6]:
features={"Pos_tags":pos_tokenize,"characters":char_tokenize,"tokenize":nltk.word_tokenize,"tokenize_stem":tokenize_stem}


In [7]:
def run_experiment_w_features(X_train,X_test,y_train,y_test,feature,n_range,theclassifiers,vectorizer=None):
    if vectorizer==None:
        vectorizer = TfidfVectorizer(
            tokenizer= features[feature],
#             cleaning itself
#             preprocessor=word_process_clean,
            ngram_range=n_range,
            use_idf=True,
            min_df=0.003,
            norm=None, )
        
    
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test=vectorizer.transform(X_test).toarray()

#     y=Y[:]
    if type(theclassifiers)!=list:
        theclassifiers=[theclassifiers]
    for theclassifier in theclassifiers:
#       initialise classifier
        clf= theclassifier()
#     create model with tranining data
        model = clf.fit(X_train, y_train)
#     predict test set
        y_preds = model.predict(X_test)
#     create the report
        report = classification_report( y_test, y_preds )
    
#     find name of the classifier for printing
        match=re.search(r"\.([A-z]*)'>",str(theclassifier))
        match=match.group(1)

        result_text="\033[1m Performance report of \033[0m \033[92m" + feature +"\033[0m "
        count=0
        for i in range(n_range[0],n_range[1]+1):
            if count!=0:
                result_text+=" and "
            result_text=result_text+"\033[91m"+str(i)+"-gram\033[0m"
            count+=1
        result_text+= " with \033[94m"+match+"\033[0m"
    #     print("Performance report of {} {}-gram".format(feature,n_range[1]))
        print (result_text)
#         prnt_scores(report)
        print(report)
        print(accuracy_score( y_test, y_preds))


In [8]:
# pre_process_clean(a_text)
for a_dict in [article_text_dict_positive,iter1_BBC_text_dict_neg,iter2_BBC_text_dict_neg,
iter1_CNN_neg_text]:
    for key,value in a_dict.items():
        a_dict[key]=pre_process_stopwords(pre_process_clean(value))


In [9]:
# for key,value in article_text_dict_positive.items():
#     print (key,value)
#     break
article_text_dict_positive.keys()

dict_keys(['3815', '3575', '3392', '3241', '3156', '3012', '2910', '2803', '2757', '2643', '2635', '2599', '2582', '2556', '2439', '2438', '2437', '2345', '2339', '2330', '2326', '2269', '2268', '2259', '2224', '2209', '2195', '2187', '2162', '2148', '2123', '2117', '2112', '2109', '2108', '2103', '2098', '2084', '2081', '2071', '2067', '2061', '2058', '2051', '2044', '2043', '2021', '1946', '1937', '1929', '1920', '1900', '1897', '1838', '1835', '1818', '1814', '1808', '1804', '1792', '1764', '1733', '1728', '1705', '1701', '1695', '1693', '1677', '1676', '1665', '1649', '1645', '1624', '1623', '1603', '1583', '1580', '1570', '1566', '1555', '1544', '1490', '1489', '1453', '1446', '1442', '1432', '1416', '1403', '1387', '1358', '1315', '1308', '1302', '1297', '1217', '1199', '1086', '1064', '1036', '1030', '1002', '998', '977', '975', '954', '934', '873', '842', '838', '795', '794', '744', '730', '685', '671', '636', '433', '313', '303', '283', '281', '263', '256', '254', '234', '3878

In [10]:
pos_examp=[]
neg_examp_train=[]
neg_examp_test=[]

for artc in article_text_dict_positive.values():
    if len(artc)>200:
        pos_examp.append((artc,1))

for artc in iter2_BBC_text_dict_neg.values():
    if len(artc)>200:
        neg_examp_train.append((artc,0))

for artc in iter1_CNN_neg_text.values():
    if len(artc)>200:
        neg_examp_test.append((artc,0))




# combine positive and negative samples then shuffle
# XY=pos_examp+neg_examp_train
random.seed(a=2)
random.shuffle(pos_examp)
random.shuffle(neg_examp_train)
random.shuffle(neg_examp_test)

percentage=0.8
cut_point=int(len(pos_examp)*percentage)
train_pos=pos_examp[:cut_point]
test_pos=pos_examp[cut_point:]

cut_point=int(len(neg_examp_train)*percentage)
train_neg=neg_examp_train[:cut_point]
cut_point=int(len(neg_examp_train)*percentage)
test_neg=neg_examp_train[cut_point:]

print("train_pos",len(train_pos),"test_pos",len(test_pos))
print("train_neg",len(train_neg),"test_neg",len(test_neg))

XY_train=train_pos+train_neg
random.shuffle(XY_train)
X_train=[k[0] for k in XY_train]
Y_train=[k[1] for k in XY_train]

XY_test=test_pos+test_neg
random.shuffle(XY_test)
X_test=[k[0] for k in XY_test]
Y_test=[k[1] for k in XY_test]

print("X_train",len(X_train),"Y_train",len(Y_train),"X_test",len(X_test),"Y_test",len(Y_test))



train_pos 828 test_pos 208
train_neg 824 test_neg 207
X_train 1652 Y_train 1652 X_test 415 Y_test 415


In [11]:
classifiers_list=[LinearSVC,MultinomialNB,RandomForestClassifier,
                  AdaBoostClassifier]
tryfeatures={"tokenize":nltk.word_tokenize,"tokenize_stem":tokenize_stem,"Pos_tags":pos_tokenize,"characters":char_tokenize}



In [20]:
len(neg_examp_train)

1031

In [ ]:
for clasifier in classifiers_list:
    for feature in tryfeatures:
        for i in range(1,4):
            for k in range(i,4):
                run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],feature,(i,k),clasifier)


In [ ]:
slow_classifiers_list=[DecisionTreeClassifier,GaussianNB,MLPClassifier]
for clasifier in slow_classifiers_list:
    for feature in tryfeatures:
        for i in range(1,4):
            run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],feature,(i,i),clasifier)

In [ ]:
myfile=open("results/4-(BBC-CNN).txt")
lines=myfile.readlines()
myfile.close()

In [ ]:
print(*lines[9:18])
print(*lines[18:27])
print(lines[18].strip()[23:].split(" ")[0])
print(lines[9].strip()[23:].split(" and "))
print(lines[9].strip()[23:].split(" ")[0])

In [ ]:
for i in range(0,len(lines),9):
    one_line=lines[i:i+9]
    first_line=one_line[0].strip()[23:]
    ngram=[]
    ngrams=first_line.split(" and ")
    if len(ngrams)==2:
        clasifier=first_line.split("with")[1].strip()
        ngram.append(ngrams[0][-6:])
        ngram.append(ngrams[1][:6])
        first_line=first_line.split(" ")

    elif len(ngrams)==3:
        clasifier=first_line.split("with")[1].strip()
        ngram.append(ngrams[0][-6:])
        ngram.append(ngrams[1])
        ngram.append(ngrams[2][:6])
        first_line=first_line.split(" ")
    else:
        first_line=first_line.split(" ")
        ngram.append(first_line[1])
        clasifier=first_line[3]
    
    feature=first_line[0]
    precision=one_line[6].split(" ")[9]
    recall=one_line[6].split(" ")[15]
    f1score=one_line[6].split(" ")[21]
    accuray=one_line[8].strip()
    for i,k in enumerate(ngram):
        ngram[i]=k.split("-")[0]
#     +("%.2f" % round(float(accuray),2))
    if feature=="characters" and (ngram[0]=="2" ):
        print("\hline")
        print(classifiers_names[clasifier]+" & "+",".join(ngram)+"-gram"+" & "+precision+" & "+recall+" & "+f1score+" \\\\")
#         print(feature,ngram,clasifier,accuray)


In [ ]:
\hline
Linear SVM (SVC) & 1-gram & 0.95 & 0.95 & 0.95 & 0.97 & 0.96 & 0.96 &  \\
\hline
Linear SVM (SVC) & 1,2-gram & 0.96 & 0.96 & 0.96 & 0.98 & 0.98 & 0.98 &  \\
\hline
Linear SVM (SVC) & 1,2,3-gram & 0.97 & 0.97 & 0.97 & 0.98 & 0.98 & 0.98 &  \\
\hline
Multinomial NB & 1-gram & 0.95 & 0.94 & 0.94 & 0.97 & 0.96 & 0.96 &  \\
\hline
Multinomial NB & 1,2-gram & 0.96 & 0.96 & 0.96 & 0.98 & 0.97 & 0.97 &  \\
\hline
Multinomial NB & 1,2,3-gram & 0.96 & 0.96 & 0.96 & 0.98 & 0.97 & 0.97 &  \\
\hline
Random Forest & 1-gram & 0.90 & 0.89 & 0.89 & 0.93 & 0.92 & 0.92 &  \\
\hline
Random Forest & 1,2-gram & 0.91 & 0.90 & 0.90 & 0.94 & 0.93 & 0.93 &  \\
\hline
Random Forest & 1,2,3-gram & 0.90 & 0.89 & 0.89 & 0.92 & 0.90 & 0.90 &  \\
\hline
AdaBoost & 1-gram &0.88 & 0.88 & 0.88 & 0.26 & 0.46 & 0.33 &  \\
\hline
AdaBoost & 1,2-gram &0.88 & 0.88 & 0.88 & 0.26 & 0.46 & 0.33 &  \\
\hline
AdaBoost & 1,2,3-gram &0.88 & 0.88 & 0.88 & 0.26 & 0.46 & 0.33 &  \\
\hline
Decision Tree & 1-gram &0.80 & 0.79 & 0.79 & 0.92 & 0.91 & 0.91 &  \\
\hline
Gaussian NB & 1-gram &0.89 & 0.88 & 0.88 &  0.98 & 0.97 & 0.97 &  \\
\hline
Multi-layer Perc. (MLP) & 1-gram &0.97 & 0.97 & 0.97 & 0.27 & 0.51 & 0.35 &  \\

In [ ]:
list(range(0,10,9))

In [ ]:
one_line=BBC_BBC_lines[9:9+9]

In [ ]:
one_line[0].strip()[23:].split("and")

In [ ]:
for line in lines:
    if "Forest" in line:
        print (line.split("and"))

In [ ]:
classifiers_names={"LinearSVC":"Linear SVM (SVC)","MultinomialNB":"Multinomial NB","RandomForestClassifier":"Random Forest",
                  "AdaBoostClassifier":"AdaBoost","DecisionTreeClassifier":"Decision Tree","GaussianNB":"Gaussian NB","MLPClassifier":"Multi-layer Perc. (MLP)"}

In [ ]:
from nltk.corpus import stopwords
